[理解学习Python装饰器(Decorator)](https://www.jianshu.com/p/ee82b941772a)

---

Python装饰器看起来类似Java中的注解，然鹅和注解并不相同，不过同样能够实现面向切面编程。

想要理解Python中的装饰器，不得不先理解闭包（closure）这一概念。

# 闭包


看看维基百科中的解释：

> 在计算机科学中，闭包（英语：Closure），又称词法闭包（Lexical Closure）或函数闭包（function closures），是引用了自由变量的函数。这个被引用的自由变量将和这个函数一同存在，即使已经离开了创造它的环境也不例外。



In [3]:
# print_msg是外围函数
def print_msg():
    msg = "外围函数变量"

    # printer是嵌套函数
    def printer():
        print('嵌套函数调用: ',msg)
    
    # 返回内部函数
    return printer


closure = print_msg()  # 这里获得的就是一个闭包

closure()  # 输出 "嵌套函数调用:  外围函数变量"


嵌套函数调用:  外围函数变量


msg是一个局部变量，在print_msg函数执行之后应该就不会存在了。但是嵌套函数引用了这个变量，将这个局部变量封闭在了嵌套函数中，这样就形成了一个闭包。

结合这个例子再看维基百科的解释，就清晰明了多了。闭包就是引用了自有变量的函数，这个函数保存了执行的上下文，可以脱离原本的作用域独立存在。

下面来看看Python中的装饰器。

# 装饰器Decorator

## 普通装饰器
一个普通的装饰器一般是这样：

In [11]:
import functools


def log(func):
    print('>>>>>>>>>>>>进入log函数>>>>>>>>>>>>')
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('==========进入wrapper函数==========')
        print('调用 %s():' % func.__name__)
        print('参数 = {}'.format(*args))
        print('==========退出wrapper函数==========')
        return func(*args, **kwargs)
    print('>>>>>>>>>>>>退出log函数>>>>>>>>>>>>')
    return wrapper


这样就定义了一个打印出方法名及其参数的装饰器。

调用之：

In [20]:
@log
def test(p):
    print('----------进入Test函数----------')
    print(test.__name__ + " 的参数是: " + p)
    print('----------退出Test函数----------')

def kkk(p):
    print('----------进入kkk函数----------')
    print(kkk.__name__ + " 的参数是: " + p)
    print('----------退出kkk函数----------')
    

print('\n测试1：\n')
    
test("abc123")  # 测试装饰器

print('\n测试2：\n')

kkk('456')  # 测试一般函数

>>>>>>>>>>>>进入log函数>>>>>>>>>>>>
>>>>>>>>>>>>退出log函数>>>>>>>>>>>>

测试1：

==========进入wrapper函数==========
调用 test():
参数 = abc123
==========退出wrapper函数==========
----------进入Test函数----------
test 的参数是: abc123
----------退出Test函数----------

测试2：

----------进入kkk函数----------
kkk 的参数是: 456
----------退出kkk函数----------


### 对比直接函数调用

装饰器在使用时，用了`@`语法，让人有些困扰。

其实，**装饰器只是个方法**，与下面的调用方式没有区别：

In [24]:
def log(func):
    print('>>>>>>>>>>>>进入log函数>>>>>>>>>>>>')
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('==========进入wrapper函数==========')
        print('调用 %s():' % func.__name__)
        print('参数 = {}'.format(*args))
        print('==========退出wrapper函数==========')
        return func(*args, **kwargs)
    print('>>>>>>>>>>>>退出log函数>>>>>>>>>>>>')
    return wrapper

def test(p):
    print('----------进入Test函数----------')
    print(test.__name__ + " 的参数是: " + p)
    print('----------退出Test函数----------')
    

print("Step.1")
wrapper = log(test)
print("Step.2")
wrapper("abc123456")

Step.1
>>>>>>>>>>>>进入log函数>>>>>>>>>>>>
>>>>>>>>>>>>退出log函数>>>>>>>>>>>>
Step.2
==========进入wrapper函数==========
调用 test():
参数 = abc123456
==========退出wrapper函数==========
----------进入Test函数----------
test 的参数是: abc123456
----------退出Test函数----------


`@语法`只是将函数传入装饰器函数，并无神奇之处。

值得注意的是`@functools.wraps(func)`，这是`python提供的装饰器`。它能把原函数的`元信息`拷贝到`装饰器里面的 func 函数中`。函数的元信息包括`docstring`、`name`、`参数列表`等等。

可以尝试去除`@functools.wraps(func)`，你会发现`test.__name__`的输出变成了`wrapper`。(并木有发生变化？！)

In [1]:
import functools

def log(func):
    print('>>>>>>>>>>>>进入log函数>>>>>>>>>>>>')
    #@functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('==========进入wrapper函数==========')
        print('调用 %s():' % func.__name__)
        print('参数 = {}'.format(*args))
        print('==========退出wrapper函数==========')
        return func(*args, **kwargs)
    print('>>>>>>>>>>>>退出log函数>>>>>>>>>>>>')
    return wrapper

def test(p):
    print('----------进入Test函数----------')
    print(test.__name__ + " 的参数是: " + p)
    print('----------退出Test函数----------')
    

print("Step.1")
wrapper = log(test)
print("Step.2")
wrapper("abc123456")

Step.1
>>>>>>>>>>>>进入log函数>>>>>>>>>>>>
>>>>>>>>>>>>退出log函数>>>>>>>>>>>>
Step.2
==========进入wrapper函数==========
调用 test():
参数 = abc123456
==========退出wrapper函数==========
----------进入Test函数----------
test 的参数是: abc123456
----------退出Test函数----------


## 带参数的装饰器

装饰器允许传入参数，一个携带了参数的装饰器将有三层函数，如下所示：



In [8]:
import functools

def log_with_param(text):
    print('>>>>>>>>>>>>进入log函数>>>>>>>>>>>>')
    def decorator(func):
        print('----------进入decorator函数----------')
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print('==========进入wrapper函数==========')
            print('call %s():' % func.__name__)
            print('args = {}'.format(*args))
            print('log_param = {}'.format(text))
            print('==========退出wrapper函数==========')
            return func(*args, **kwargs)
        print('----------退出decorator函数----------')
        return wrapper
    
    print('>>>>>>>>>>>>退出log函数>>>>>>>>>>>>')
    return decorator
    
@log_with_param("log的参数")
def test_with_param(p):
    print('::::::::::进入test函数::::::::::')
    print('函数名字 : ',test_with_param.__name__)
    print('参数     : ', p)
    print('::::::::::退出test函数::::::::::')

print('Step.1')
test_with_param("1234")
print('Step.2')

>>>>>>>>>>>>进入log函数>>>>>>>>>>>>
>>>>>>>>>>>>退出log函数>>>>>>>>>>>>
----------进入decorator函数----------
----------退出decorator函数----------
Step.1
==========进入wrapper函数==========
call test_with_param():
args = 1234
log_param = log的参数
==========退出wrapper函数==========
::::::::::进入test函数::::::::::
函数名字 :  test_with_param
参数     :  1234
::::::::::退出test函数::::::::::
Step.2


### 对比直接函数调用
看到这个代码是不是又有些疑问，内层的`decorator函数`的参数`func`是怎么传进去的？和上面一般的装饰器不大一样啊。

其实道理是一样的，将其`@`语法去除，恢复函数调用的形式一看就明白了：

In [9]:
def log_with_param(text):
    print('>>>>>>>>>>>>进入log函数>>>>>>>>>>>>')
    def decorator(func):
        print('----------进入decorator函数----------')
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print('==========进入wrapper函数==========')
            print('call %s():' % func.__name__)
            print('args = {}'.format(*args))
            print('log_param = {}'.format(text))
            print('==========退出wrapper函数==========')
            return func(*args, **kwargs)
        print('----------退出decorator函数----------')
        return wrapper
    
    print('>>>>>>>>>>>>退出log函数>>>>>>>>>>>>')
    return decorator
    
#@log_with_param("log的参数")  # 这里注释了
def test_with_param(p):
    print('::::::::::进入test函数::::::::::')
    print('函数名字 : ',test_with_param.__name__)
    print('参数     : ', p)
    print('::::::::::退出test函数::::::::::')



print('Step.1')
# 传入装饰器的参数，并接收返回的decorator函数
decorator = log_with_param("log的参数")

print('Step.2')
# 传入test_with_param函数
wrapper = decorator(test_with_param)

print('Step.3')
# 调用装饰器函数
wrapper("1234")
print('Step.4')

Step.1
>>>>>>>>>>>>进入log函数>>>>>>>>>>>>
>>>>>>>>>>>>退出log函数>>>>>>>>>>>>
Step.2
----------进入decorator函数----------
----------退出decorator函数----------
Step.3
==========进入wrapper函数==========
call test_with_param():
args = 1234
log_param = log的参数
==========退出wrapper函数==========
::::::::::进入test函数::::::::::
函数名字 :  test_with_param
参数     :  1234
::::::::::退出test函数::::::::::
Step.4


---

至此，装饰器这个有点费解的特性也没什么神秘了。

装饰器这一语法体现了Python中**函数是第一公民**，函数是对象、是变量，可以作为参数、可以是返回值，非常的灵活与强大。

Python中引入了很多函数式编程的特性，需要好好学习与体会。